In [ ]:
import os

import numpy as np

import sympy as sp
from sympy import cot, tan, acot, atan, pi

from symr.metrics import compute_complexity, compute_shannon_diversity

In [ ]:
def save_expressions(expressions, variables, filename="temp.csv", support=None, comments=None):
    """
    save csv with columns
    expression, support, number_variables, variables, comments
    """ 

    my_lines = ["expression, support, number_variables, variables, complexity, diversity, comments\n"]
    
    for index, (expr, my_vars) in enumerate(zip(expressions, variables)):
        try:
            sympy_expr = sp.sympify(expr)
        except:
            assert False, (f"{expr} didn't work")
            
        if sympy_expr.has(cot, acot):
            # https://github.com/sympy/sympy/issues/6672
            # https://en.wikipedia.org/wiki/Trigonometric_functions
            cots = {cot_term: 1/tan(*cot_term.args) for cot_term in sympy_expr.atoms(cot)}
            acots = {acot_term: pi/2 - atan(*acot_term.args) for acot_term in sympy_expr.atoms(acot)}
            sympy_expr = sympy_expr.subs(cots).subs(acots)
        
        number_variables = my_vars.count(",") + 1
        if support == None:
            my_support = ""
            for ii in range(number_variables):
                my_support += "-5 5 "
        else:
            my_support = support[index]
            
        if comments == None:
            my_comment = ""
        else:
            my_comment = comments[index]
            
        # test expressions valid within support
        
        supports = my_support.split(" ")
        support_vars = my_vars.split(",")
        
        try:
            expr_function = sp.lambdify(my_vars, expr=sympy_expr)
            my_input = []
            for jj in range(len(support_vars)):
                low = float(supports[2*jj])
                high = float(supports[2*jj+1])
                step_size = (high-low)/1000
                my_input.append(np.arange(low, high, step_size))
            
            output = expr_function(*my_input)
            all_ok = True
            for bad_out in [float("inf"), -float("inf"), float("nan")]:
                
                if (bad_out == output).any():
                    #print(f"{bad_out} detected")
                    
                    all_ok = False
                    
            assert all_ok, "not all OK"
            
        except:
            import pdb; pdb.set_trace()
            print("warning, expression invalid within support")
            print(f"support: {my_support}, expression: {sympy_expr}")
        
            
        my_vars = my_vars.replace(","," ")
        
        my_complexity = compute_complexity(sp.simplify(sympy_expr))
        my_diversity = compute_shannon_diversity(sp.simplify(sympy_expr))
            
        my_lines.append(f"{sympy_expr}, {my_support}, {number_variables}, {my_vars}, "\
                        f"{my_complexity}, {my_diversity}, {my_comment}\n")
    
    
        
    with open(filename, "w") as f:
        f.writelines(my_lines)
            
    if os.path.exists(filename):
        
        print(f"expressions saved successfuly to {filename}")
        
    return 1

In [ ]:
# Make Strogatz benchmark and save
# equation, support, number_variables, variables, comments

strogatz_variables = ["x, y", \
            "x, y",\
            "theta, phi",\
            "theta, phi",\
            "v, theta",\
            "v, theta",\
            "x, y",\
            "x, y",\
            "x, y",\
            "x, y",\
            "theta, phi",\
            "theta, phi",\
            "x, y",\
            "x, y"]

strogatz_eqns = ["20 - x - (x*y) / (1+0.5*x**2)",\
                "10 - (x*y)/(1+0.5*x**2)",\
                "0.5 * sin(theta - phi) - sin(theta)",\
                "0.5 * sin(phi - theta) - sin(phi)",\
                "0.05 * v**2 - sin(theta)",\
                "v - cos(theta) / v",\
                "3 * x - 2 * x * y - x**2",\
                "2 * y - x * y - y**2",\
                "x * (4 - x- y / (1+x))",\
                "y * (x / (1+x) - 0.075 *y)",\
                "1/acot(phi) * cos(theta)",\
                "(cos(phi)**2 + 0.1* sin(phi)**2) * sin(theta)",\
                "10 * (y - 1/3 * (x**3-x))",\
                "-1/10 * x"]

strogatz_comments =["bacterial respiration",\
                   "bacterial respiration",\
                   "bar magnets",\
                   "bar magnets",\
                   "glider",\
                   "glider",\
                   "Lotka-Volterra",\
                   "Lotka-Volterra",\
                   "predator prey",\
                   "predator prey",\
                   "shear flow",\
                   "shear flow",\
                   "van der Pol",\
                   "van der Pol"]

save_expressions(strogatz_eqns, strogatz_variables, filename="../data/strogatz.csv", comments=strogatz_comments)

In [ ]:
nguyen_variables = ["x",\
                   "x",\
                   "x",\
                   "x",\
                   "x",\
                   "x",\
                   "x",\
                   "x",\
                   "x, y",\
                   "x, y",\
                   "x, y",\
                   "x, y"]

nguyen_expressions = ["x**3 + x**2 + x",\
                      "x**4 + x**3 + x**2 + x",\
                      "x**5 + x**4 + x**3 + x**2 + x",\
                      "x**6 + x**5 + x**3 + x**2 + x",\
                      "sin(x**2) * cos(x) - 1",\
                      "sin(x) + sin(x + x**2)",\
                      "log(x + 1) + log(x**2+1)",\
                      
                      "sqrt(x)",\
                      "sin(x) + sin(y**2)",\
                      "2*sin(x)*cos(y)",\
                      "x**y",\
                      "x**4 - x**3 + (1/2) * y**2 - y"]

nguyen_support = ["-5 5",\
                  "-5 5",\
                  "-5 5",\
                  "-5 5",\
                  "-5 5",\
                  "-5 5",\
                  "-0.99 5",\
                  "-5 5",\
                  "-5 5 -5 5",\
                  "-5 5 -5 5",\
                  "-5 5 -5 5",\
                  "-5 5 -5 5",\
                 ]

nguyen_comments = [f"Nguyen-{ii}" for ii in range(len(nguyen_expressions))]


save_expressions(nguyen_expressions, nguyen_variables, filename="../data/nguyen.csv", support=nguyen_support, comments=nguyen_comments)

In [ ]:
ab_variables = ["x",\
               "x",\
               "x",\
               "x",\
               "x",\
               "x",\
               "x",\
               "x"]

ab_expressions = ["sin(x * exp(x))",\
                  "x + log(x**4)",\
                  "1+x*sin(1/x)",\
                  "sqrt(x**3) * log(x**2)",\
                  "(x+x**3) / (1+x*cos(x**2))",\
                  "x / (sqrt(x**2 + sin(x)))",\
                  "cos((x+sin(x))/ (x**3+x*log(x**2)))",\
                  "(exp(x) * (1+sqrt(1+x) + cos(x**2)))/ x**2"]

ab_support = ["-5 5",\
                  "-5 5",\
                  "-5 5",\
                  "-5 5",\
                  "-5 5",\
                  "-5 5",\
                  "-5 5",\
                  "-5 5",\
                 ]

ab_comments = [f"AB-{ii}" for ii in range(len(ab_expressions))]


save_expressions(ab_expressions, ab_variables, filename="../data/ab_complex.csv", support=ab_support, comments=ab_comments)

In [ ]:
# get the Feynman equations if not already downloaded
! wget https://space.mit.edu/home/tegmark/aifeynman/FeynmanEquations.csv data/FeynmanEquations.csv

In [ ]:
filename = "../data/FeynmanEquations.csv"

with open(filename, "r") as f:
    lines = f.readlines()
    
feynman_variables = []
feynman_expressions = []
feynman_support = []
feynman_comments = []

replacements = [("alpha", "alfa"),("beta", "betta"), ("gamma","gama")]

for line in lines[1:]:
    
    lines_list = line.split(",")
    
    variable_index = 5
    my_variable = lines_list[variable_index]
    my_support = f"{lines_list[variable_index+1]} {lines_list[variable_index+2]}"
    my_variables = ""
   
    while my_variable != "" and variable_index <= 32:
        my_variable = my_variable.replace('"','')
        for (rep, sub) in replacements:
            my_variable = my_variable.replace(rep, sub)
            
        my_variables = f"{my_variables}{my_variable}, "
        
        variable_index += 3
        my_variable = lines_list[variable_index]
         
        my_support = f"{my_support} {lines_list[variable_index+1]} {lines_list[variable_index+2]}"
        
        
    my_support = my_support[:-2]
    my_variables = my_variables[:-2]
    
    feynman_comments.append(f"Feynman-{lines_list[0]} equation number {lines_list[1]}")
    feynman_variables.append(my_variables)
    feynman_support.append(my_support)
    
    my_expression = lines_list[3].replace('"', "")
    for (rep, sub) in replacements:
        my_expression = my_expression.replace(rep, sub)
       
    feynman_expressions.append(my_expression)
    

In [ ]:
save_expressions(feynman_expressions, \
                 feynman_variables, \
                 filename="../data/feynman.csv", \
                 support=feynman_support, \
                 comments=feynman_comments)